**Dependencies**

In [ ]:
# Installing python dependencies
%pip install openai jsonlines

In [ ]:
# Import Libraries

import json
from openai import OpenAI

**Module**

In [ ]:
%pip install nbformat

**Authentication**

In [ ]:
# Initializing GPT Api
api_key = ""
client = OpenAI(api_key=api_key)

**Dataset**

In [ ]:
#Trip dataset
%run dataset.ipynb

print(data) 

In [ ]:
#Converting the database to a suitable format\
import jsonlines

jsonl_filename = "output.jsonl"

#save as JSONL (Json line file)
with jsonlines.open(jsonl_filename,"w") as writer:
    writer.write_all(data)

In [ ]:
# Insert the training data into context
file = client.files.create(
  file = open(jsonl_filename, "rb"),
  purpose = "fine-tune"
)


# Extracting the dataset id
dataset = file.id
print(dataset)

**Training our model**

In [ ]:
# Starting a fine-tuning job
trained_model = client.fine_tuning.jobs.create(
  training_file=dataset, 
  model="gpt-3.5-turbo",
hyperparameters={
    "n_epochs":1
  }
)

# Retrieving The model Training Status
jobstatus = client.fine_tuning.jobs.retrieve(trained_model.id)
print(jobstatus)

In [ ]:
# Retrieving The model Training Status
jobstatus = client.fine_tuning.jobs.retrieve(trained_model.id)
print(jobstatus)

**Extracting Model**

In [ ]:
# Retrieving The fine-tuned model id
fine_tuned_model = jobstatus.fine_tuned_model
print(fine_tuned_model)

Interacting with ai


In [ ]:
# Generate the response
def answer(question):

    prompt = [{"role": "system", "content": "Marv is a rude chatbot that is also blunt."}, {"role": "user", "content": question}]
    response = client.chat.completions.create(
        model=fine_tuned_model,
        messages=prompt 
        )
    
    response_msg = response.choices[0].message.content

    return response_msg

In [ ]:
#prompt for input from user
country = input("Country: ")
state = input("State: ")
day = input("Number of day: ")

command = f"country: {country}, state: {state}, day: {day}"

# Generate the response
output = answer(command)
print(output)